In [15]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
import torch
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
authors = pd.read_csv("../../data/authors.csv")
authors = authors[["comment", "username"]]
authors = shuffle(authors, random_state=42)
authors.reset_index(inplace=True)
authors.drop("index", axis=1, inplace=True)

In [4]:
authors

,comment,username
0,Essa galera ta na 2° internacional ainda kkkkk,ToguroSenpai
1,"Rapaz, vcs falas as coisas, e ninguém pode dis...",squiercg
2,Levonhardu para deputado federal!,Manada_2
3,Basement dwellers,BluePirate89
4,Frase mentirosa. Sempre a mesma coisa. Sempre ...,9-Cortes
...,...,...
23943,Eu não gosto do Kim mas o fato do Monark chama...,uziel7
23944,"Me parecem boas causas ao Partido ""A Unidade P...",Manperapp
23945,Ele está acostumado a engolir a gosminha molha...,Supermunch2000
23946,Bolsonaro depois de receber o dinheiro da milí...,BluePirate89


In [16]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
le = LabelEncoder()
ids = le.fit_transform(authors.username)
labels = dict(zip(le.classes_, range(len(le.classes_))))

Downloading: 100%|██████████| 205k/205k [00:00<00:00, 341kB/s] 
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 477B/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 78.8kB/s]
Downloading: 100%|██████████| 43.0/43.0 [00:00<00:00, 16.2kB/s]
Downloading: 100%|██████████| 647/647 [00:00<00:00, 389kB/s]


In [17]:
y = [labels[label] for label in authors['username']]
print("Quantidade de comentários:", len(y))

Quantidade de comentários: 23948


In [18]:
texts = [tokenizer.encode(text, 
                padding='max_length', 
                max_length = 512, 
                truncation=True, 
                return_tensors="pt") 
            for text in authors['comment']]

In [ ]:
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert_embeddings = list()

In [ ]:
print(texts)

In [40]:
for i, text in enumerate(texts):
    with torch.no_grad():
        outs = model(text)
        encoded = outs[0][0, 1:-1]
        # with open('document.csv','a') as fd:
        #     fd.write(myCsvRow)
    d = {"comment": authors.loc[i, "comment"], 
    "username": authors.loc[i, "username"],
    "embedding": encoded}
    break

    if i%100 == 0: print(i)

{'comment': 'Essa galera ta na 2° internacional ainda kkkkk', 'username': 'ToguroSenpai', 'embedding': tensor([[ 1.5931e-02, -1.1629e-01,  9.4360e-01,  ...,  1.2968e-01,
          4.1585e-01, -8.6580e-02],
        [ 4.9208e-01, -1.7203e-01,  1.7385e-01,  ...,  1.2356e-01,
          3.6446e-01,  1.3847e-03],
        [-2.3169e-01, -5.7046e-02,  1.5214e+00,  ..., -7.6107e-02,
          3.7596e-01, -7.2401e-01],
        ...,
        [-5.6271e-01, -1.5298e-01,  8.3737e-01,  ...,  5.6805e-01,
          8.4915e-01,  2.0434e-01],
        [-5.3157e-01, -1.0473e-01,  9.2934e-01,  ...,  4.3956e-01,
          8.3178e-01,  2.2532e-01],
        [-3.2283e-01, -1.9415e-02,  8.5392e-01,  ...,  1.7822e-01,
          6.3939e-01,  8.2012e-02]])}


In [ ]:
X = bert_embeddings
y = authors["username"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [91]:
#X_train2 = [np.array(x.reshape(1,-1)) for x in X_train]
#len(X_train2)

67

In [ ]:
X_train2 = np.array([np.array(x.reshape(1,-1).flatten()) for x in X_train])
X_train2.shape

(67, 391680)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train2, y_train)

DecisionTreeClassifier()

In [ ]:
X_test2 = np.array([np.array(x.reshape(1,-1).flatten()) for x in X_test])
X_test2.shape

(34, 391680)

In [ ]:
y_pred = clf.predict(X_test2)

In [ ]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

MidnightRider00       0.68      0.68      0.68        22
        TheGza1       0.42      0.42      0.42        12

       accuracy                           0.59        34
      macro avg       0.55      0.55      0.55        34
   weighted avg       0.59      0.59      0.59        34

